### Loading the Data

In [1]:
import pandas as pd
import numpy as np

In [2]:
lineup_dat = pd.read_csv("Game_Lineup.txt", "\t")
play_dat = pd.read_csv("Play_by_Play.txt", "\t")

### Cleaning the data set specified in the instructions

In [3]:
# To properly sort the events in a game, use the following sequence of sorted columns: 
# Period (ascending), PC_Time (descending), WC_Time (ascending), Event_Num (ascending)
play_dat = play_dat.sort_values(["Period", "PC_Time", "WC_Time","Event_Num"], axis = 0, 
                 ascending = [True, False, True, True], inplace = False)

### Merging player data with game by game data

In [4]:
# in most scenarios, this is the Team_id associated with the Person1 column. 
# However, there are instances when this is not the case. To
# accurately and consistently identify a player’s team, 
# we suggest merging with the Game_Lineup dataset on the Person1 and Person2 columns.

# First get a df of the team_id for each player_id

all_players = list(set(lineup_dat["Person_id"]))

my_list = []
for i in range(0, len(all_players)):
    
    x = lineup_dat[lineup_dat["Person_id"] == all_players[i]]["Team_id"]
    x = list(x)
    my_list.append(x[0])

len(my_list)

player_team_df = pd.DataFrame(my_list, all_players).reset_index(drop = False)
player_team_df.columns = ["Person_id", "Team_id"]

In [5]:
# Get the accurate team_id to correspond with each player_id of each row in play_dat
team_id_list = []
for i in play_dat["Person1"]:
    x = player_team_df[player_team_df["Person_id"] == i]["Team_id"]
    if x.empty == False:
        x = list(x)
        team_id_list.append(x[0])
    else:
        team_id_list.append("N/A")

In [6]:
# Check to see if it worked, this shows there were 14,697 times where the team_id did not correspond to the player_id
(play_dat["Team_id"] == team_id_list).value_counts()

True     23192
False    14697
Name: Team_id, dtype: int64

In [7]:
play_dat["Team_id_updated"] = team_id_list

In [8]:
event_codes = pd.read_csv("Event_Codes.txt", "\t")
event_codes[event_codes["Event_Msg_Type"] == 10]

,Event_Msg_Type,Action_Type,Event_Msg_Type_Description,Action_Type_Description
266,10,0,Jump Ball,


In [9]:
event_descriptiion = []
event_msg_type = list(play_dat["Event_Msg_Type"])
action_type = list(play_dat["Action_Type"])

In [10]:
number_of_rows = len(action_type)

description_columns = []
for row_number in range(0,number_of_rows):
    
    # get from play_dat the event and action
    event = event_msg_type[row_number]
    action = action_type[row_number]
    
    # subset the event_codes table to get the descriptions for the respective event and action
    subset = event_codes[(event_codes["Event_Msg_Type"] == event) & (event_codes["Action_Type"] == action)]
    
    event_msg_type_description = list(subset["Event_Msg_Type_Description"])[0]
    action_type_description = list(subset["Action_Type_Description"])[0]

    description_columns.append([event_msg_type_description,action_type_description])
    

In [11]:
event_msg_type_result = [description[0] for description in description_columns]
action_type_result = [description[1] for description in description_columns]

In [12]:
play_dat["event_msg_type_desc"] = event_msg_type_result
play_dat["action_type_desc"] = action_type_result

In [13]:
play_dat = play_dat[['Game_id', 'Event_Num', 
                    'Event_Msg_Type', 'Action_Type',
                    'event_msg_type_desc','action_type_desc',
       'Team_id_updated', 'Period', 'WC_Time', 'PC_Time', 'Option1', 'Option2',
       'Option3', 'Team_id', 'Person1', 'Person2', 'Person3', 'Team_id_type',
       'Person1_type', 'Person2_type', 'Person3_type']]

In [14]:
play_dat[play_dat["Team_id_updated"] == "N/A"]["event_msg_type_desc"].value_counts()

Rebound                                     1361
Timeout                                      903
Stoppage                                     452
Start Period                                 332
End Period                                   332
Instant Replay                               183
Turnover                                     110
Final Box Printed                             82
Violation                                     21
Foul                                           8
Name: event_msg_type_desc, dtype: int64

### Function 2: The final function, which is condensed version of function one

In [15]:
def get_df2(player_id, game_id):
    
    """ Takes in the player_id and game_id and will return the df of rows consisting of only
        the times the player was in throughout the whole game """
    
    # Split up the data into 3 parts: first half, second half, ot
    first_half = play_dat[(play_dat["Game_id"] == game_id) & (play_dat["Period"] < 3)].reset_index(drop = True)
    second_half = play_dat[(play_dat["Game_id"] == game_id) & (play_dat["Period"] > 2) & (play_dat["Period"] != 5)].reset_index(drop = True)
    overtime = play_dat[(play_dat["Game_id"] == game_id) & (play_dat["Period"] == 5)].reset_index(drop = True)
    
    # Put all Data frames into a list 
    all_dfs = [first_half, second_half, overtime]
    
    # Getting df of each half then combinging it into one 
    x = [my_function(all_dfs[i],player_id) for i in range(0,len(all_dfs))]
    
    if all(ha is None for ha in x):
        pdf = pd.DataFrame()
    else:
        pdf = pd.concat(x).reset_index(drop = True)
    
    return pdf

In [16]:
def my_function(all_df, player_id):
    
    final_df = None
    
    # Gets indicies of when the player subbed in and subbed out
    subbed_out1 = all_df[(all_df["Event_Msg_Type"] == 8) & (all_df["Person1"] == player_id)].index.tolist()
    subbed_in1 =  all_df[(all_df["Event_Msg_Type"] == 8) & (all_df["Person2"] == player_id)].index.tolist()
    
    # Checks if player was even active and if they started but never subbed out
    if (len(subbed_in1) == 0) & (len(subbed_out1) == 0):
        if ((player_id in all_df.Person1.values) | (player_id in all_df.Person2.values) | (player_id in all_df.Person3.values)):
            final_df = all_df
        else:
            final_df = None
    
    # Checks to see if a player started, then subbed out but never subbed in again 
    if (len(subbed_in1) == 0) & (len(subbed_out1) == 1):
        
        # Check to see if they subbed out during a freethrow
        free_throw_indicies = []
        for index in subbed_out1:
            if (all_df.iloc[index+1][2] == 3) & (all_df.iloc[index+1][6] in list(range(10,30))):
                free_throw_indicies.append(index + 1)
            else:
                free_throw_indicies.append(index)
        
        subbed_out1 = free_throw_indicies
        final_df = all_df[0:subbed_out1[0]]
    
    # Check to see if a player subbed in, but then never subbed out again 
    if (len(subbed_in1) == 1) & (len(subbed_out1) == 0):
        
        # Disregard freethrows for when the player subbed in
        free_throw_indicies1 = []
        for index in subbed_in1:
            if (all_df.iloc[index+1][2] == 3) & (all_df.iloc[index+1][6] in list(range(10,30))):
                free_throw_indicies1.append(index)
            else:
                free_throw_indicies1.append(index)
        
        subbed_in1 = free_throw_indicies1
        final_df = all_df[subbed_in1[0]:]
    
    # Checks if player subbed in and out multiple times in a half 
    if (len(subbed_in1) > 0) & (len(subbed_out1) > 0):
        
        # Checks to see if a player started or not 
        if subbed_in1[0] > subbed_out1[0]:
            subbed_in1.append(0)
            subbed_in1.sort()
        
        # Check if player subbed out during free throws 
        free_throw_indicies = []
        for index in subbed_out1:
            if (all_df.iloc[index+1][2] == 3) & (all_df.iloc[index+1][6] in list(range(10,30))):
                free_throw_indicies.append(index+2)
            else:
                free_throw_indicies.append(index + 1)
        
        # Disregard freethrows for when the player subbed in
        free_throw_indicies1 = []
        for index in subbed_in1:
            if (all_df.iloc[index+1][2] == 3) & (all_df.iloc[index+1][6] in list(range(10,30))):
                free_throw_indicies1.append(index+2)
            else:
                free_throw_indicies1.append(index)
        
        # Update indicies
        subbed_out1 = free_throw_indicies
        subbed_in1 = free_throw_indicies1
        
        # Get df 
        subdfs = [all_df[i:j] for i,j in zip(subbed_in1, subbed_out1)]
        final_df = pd.concat(subdfs)
        
        # Check to see if the player finished out the half
        if len(subbed_in1) > len(subbed_out1):
            dat1 = all_df[subbed_in1[-1]:]
            final_df = final_df.append(dat1)
    
    return final_df


In [17]:
y = get_df2("42e0d7167f04a4ff958c6442da0e6851","ff42c1cba89fd3f3a7a6711a11d5576d")
len(y)

319

### Get_points function: Wiil take in a data frame and find the total amount of points scored

In [51]:
def get_points(df):
    
    """ Takes in a data frame and will calculate the amount of points scored, if the data frame enetered is empty
     or if there was no posession where a shot was made the function will return 0"""
    
    # Subsets data frame to only look at rows with made field goals and made free throws 
    fgs_df = df[((df["Event_Msg_Type"] == 3) & (df["Option1"] == 1)) | (df["Event_Msg_Type"] == 1)]
    
    new_1 = fgs_df["Option1"].sum()
    
    return new_1

In [52]:
####### Test 

# Gets the team id of the player
t_id = list(player_team_df[player_team_df["Person_id"] == "42e0d7167f04a4ff958c6442da0e6851"]["Team_id"])

# Subset the data to only include team_id of that player 
ortg_pos = y[y["Team_id_updated"] == t_id[0]]

## Subset the data to only look at places where points were scored 
fgs_df = ortg_pos[((ortg_pos["Event_Msg_Type"] == 3) & (ortg_pos["Option1"] == 1)) | (ortg_pos["Event_Msg_Type"] == 1)]
shots = dict(fgs_df["Option1"].value_counts())

# Put it in a list 
ha = list(shots.items())

new_l = []
for i in range(0, len(ha)):
    total = ha[i][0] * ha[i][1]
    new_l.append(total)

points = sum(new_l)

function_points = get_points(ortg_pos)

# Since both returned  the same amount that means the function works 
print(points)
print(function_points)
print(fgs_df["Option1"].sum())

75
75
75


### Get_pos function: Will take in a data frame of rows player was in and their team id and calculate number of possesions they were in for 

In [20]:
#### Look at and account for all cases listed in the prompt 

In [53]:
def get_possessions(df):
    
    '''Takes a df and computes the number of possessions a player was on the court for.'''
    
    final_free_throw = ["Free Throw 2 of 2", "Free Throw 1 of 1", "Free Throw 3 of 3"]
    num_pos = 0
    
    # iterate through each row of the dataframe
    for row in range(0,len(df["Game_id"])):
        
        # get the current team and the team of next row
        current_team = df["Team_id_updated"][row]
        next_team = ""
        final_row = False
        try:
            next_team = y["Team_id_updated"][row+1]
        except:
            next_team = current_team
            final_row = True
        
        # case (1) made field goal attempts,
        if (df["event_msg_type_desc"][row].strip() == 'Made Shot'):
            num_pos += 1
            
        # case (2) made final free throw attempt, 
        elif (df['action_type_desc'][row] in final_free_throw) and (df['Option1'][row] == 1):
            num_pos += 1
            
        # case (3) missed final free throw attempt that results in a defensive rebound,
        elif (not final_row) and (df['action_type_desc'][row] in final_free_throw) and (df['Option1'][row] != 1) and (df["event_msg_type_desc"][row+1].strip() == 'Rebound') and (current_team != next_team):
            num_pos += 1
            
        # case (4) missed field goal attempt that results in a defensive rebound, 
        elif (not final_row) and (df["event_msg_type_desc"][row].strip() == 'Missed Shot') and (df["event_msg_type_desc"][row+1].strip() == 'Rebound') and (current_team != next_team):
            num_pos += 1
            
        # case 5: Turnover or End of Period
        elif (df["event_msg_type_desc"][row].strip() == 'Turnover') or (df["event_msg_type_desc"][row].strip() == 'End Period'):
            num_pos += 1
        
        else:
            continue
            
            
    return num_pos


In [54]:
get_possessions(y)

132

### Final Function which gets offensive and defensive rating using the functions defined above

In [55]:
def get_rtgs(player_id,game_id):
    
    # Get Data Frame of the possesions they were in
    df = get_df2(player_id, game_id)
    
    if (df.empty):
        ortg = 0
        drtg = 0
    
    else:
        
        # Get the Team_ID of the player
        t_id = list(player_team_df[player_team_df["Person_id"] == player_id]["Team_id"])
    
        # Get points scored by only looking at rows where the df is equal to the player's team
        ortg_pos = df[df["Team_id_updated"] == t_id[0]]
        points_scored = get_points(ortg_pos)
    
        # Get points allowed by only looking at rows where the df is  not equal to the player's team
        drtg_pos = df[df["Team_id_updated"] != t_id[0]]
        points_allowed = get_points(drtg_pos)
    
        # Get amount of posessions they were in for 
        num_pos = get_possessions(df)
    
        # Then having points scored, allowed, and number of possesions calculate offensive and defensive rating
        ortg = (points_scored / num_pos) * 100
        drtg = (points_allowed / num_pos) * 100
    
    final = pd.DataFrame([[game_id,player_id,ortg,drtg]], columns= ["Game_ID","Player_ID","OffRtg","DefRtg"])
    
    return final

In [56]:
get_rtgs("42e0d7167f04a4ff958c6442da0e6851","ff42c1cba89fd3f3a7a6711a11d5576d")

,Game_ID,Player_ID,OffRtg,DefRtg
0,ff42c1cba89fd3f3a7a6711a11d5576d,42e0d7167f04a4ff958c6442da0e6851,56.818182,43.939394


### Getting Ratings For everyone

In [57]:
person_game = lineup_dat[["Person_id","Game_id"]].drop_duplicates().reset_index(drop = True)

In [58]:
finished_df = []
for i in range(0,len(person_game)):
    p_id = person_game["Person_id"][i]
    g_id = person_game["Game_id"][i]
    x = get_rtgs(p_id,g_id)
    finished_df.append(x)

In [59]:
final_product = pd.concat(finished_df)

In [60]:
len(final_product[final_product["OffRtg"] == 0])

760

In [61]:
# Save file as csv
final_product.to_csv("UCD_Q1_BBAL.csv", index=False)